### Disclaimer:

This is based on the tutorial provided by:
https://www.twilio.com/blog/build-whatsapp-bot-sentiment-analysis-python-twilio

---
# Chatter Trainer
---

In [1]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import pandas as pd

In [2]:
chatbot = ChatBot("Twily")
trainer = ListTrainer(chatbot)

[nltk_data] Downloading package stopwords to /home/lenovo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/lenovo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Training lists

Our three types of arbitrary conversations are documentation_topics, twilio_knowledge and classifier.

- documentation_topics: 
    - provides documentation links based on keywords found in the user's question
- twilio_knowledge: 
    - will answer predefined general questions
- classifier: 
    - has some keywords associated with a user frustration response, the bot will use these to try to de-escalate the user's frustration.

In [3]:
documentation_topics = [
("sdk", "https://www.twilio.com/docs/sms/whatsapp#sdks"),
("python helper library", "https://www.twilio.com/docs/libraries/python"),
("tutorials", "https://www.twilio.com/docs/sms/whatsapp/tutorial/ send-and-receive-media-messages-twilio-api-whatsapp"),
("autopilot", "https://www.twilio.com/docs/autopilot/channels/whatsapp"),
("contact us", "https://support.twilio.com/hc/en-us")]

# base knowledge
twilio_knowledge = [
("Twilio description", "Simply put, Twilio is a developer platform for communications.\
        Software teams use Twilio APIs to add capabilities like voice, video, and messaging \
        to their applications. This enables businesses to provide the right communications \
            experience for their customers."),
("Twilio email", "Sorry, you can submit a ticket at: https://www.twilio.com/console/support/tickets/create"),
("Twilio phone number", "We don't have a phone number for this type of account"),
("mailing address", "You can email our corporate headquarters at hello@craft.com"),
("chatterbot", "library making it easy to generate automated responses to a user's input, visit https://chatterbot.readthedocs.io/en/stable/"),
("textblob", "library for processing textual data, please visit https://textblob.readthedocs.io/en/dev/")
]

# sentiment
classifier = ["silly", "dumb", "stupid", "I'dont think so", "I don't care",
                   "do you know anything", "not good", "omg",
                   "this is bad", "not what I want", "live help",
                   "get me a rep", "I need a real person"]

### Read Dataset
- get the conversation from the `utterance` column
- get the sentiments from `context` and `promt` columns

In [4]:
df_train = pd.read_csv("./dataset/train_arabic_updated.csv")
utterances = df_train["utterance"]
original_sentiments = df_train[["context","prompt"]].drop_duplicates()
# sentiments = list(zip(sentiments["context"], sentiments["prompt"]))

In [5]:
original_sentiments

,context,prompt
0,sentimental,I remember going to the fireworks with my best...
6,afraid,i used to scare for darkness
12,proud,I showed a guy how to run a good bead in weldi...
17,faithful,I have always been loyal to my wife.
21,terrified,A recent job interview that I had made me feel...
...,...,...
78752,impressed,I was watching professional rodeo the other da...
78756,anticipating,I am waiting to see if I pass my graduate exam...
78760,afraid,My house burned down and I had to rescue my fa...
78764,sentimental,I found some pictures of my grandma in the att...


In [6]:
map_empathy={
    "joy":["excited","proud","grateful","hopeful","confident","joyful","content","prepared","anticipating"],
    "love":["caring","sentimental","trusting","faithful","nostalgic"],
    "surprise":["impressed","surprised"],
    "sadness":["sad","lonely","guilty","disappointed","devastated","embarrassed","ashamed"],
    "anger":["angry","annoyed","furious","disgusted","jealous"],
    "afraid":["terrified","fear","anxious","apprehensive"]
}
map_empathy={
    "pos":["excited","proud","grateful","hopeful","confident","joyful","content","prepared","anticipating",
          "caring","sentimental","trusting","faithful","nostalgic","impressed","surprised"],
    
    "neg":["sad","lonely","guilty","disappointed","devastated","embarrassed","ashamed",
           "angry","annoyed","furious","disgusted","jealous","terrified","fear","anxious","apprehensive"],
}
data =[]
for reduced_empathy in map_empathy:
    for empathy in map_empathy[reduced_empathy]:
        data.append([reduced_empathy, empathy])
  
# Create the pandas DataFrame
df_map_empathy = pd.DataFrame(data, columns = ['empathy', 'context'])


In [7]:
df_map_empathy.head()

,empathy,context
0,pos,excited
1,pos,proud
2,pos,grateful
3,pos,hopeful
4,pos,confident


In [8]:
df_mapped_empathy = pd.merge(original_sentiments, df_map_empathy, how="inner" ,on='context' )
sentiments = list(zip(df_mapped_empathy["empathy"], df_mapped_empathy["prompt"]))

### Training iterators

Below the training list we will write three list iterators, each with a pre-formatted question/answer. 
Each loop calls the trainer instance and the train() method, passing the name of the list as an argument. 
This will generate conversations using the information found in each list.

In [9]:
trainer.train(utterances[:50])

for i in classifier:
    trainer.train([
        f"{i}",
        "I am sorry you feel that way, please ask the question again"
    ])

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%


### Corpus file export

Finally we export our trainer instance to a JSON file using ChatterBot's .export_for_training() method, which takes an argument in the form of the JSON filename.

In [14]:
trainer.export_for_training('EnglishChatBot.json')

---
# Classifier
---

### TextBlob

"TextBlob provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more." ~ TextBlob documentation


This application will mostly rely on TextBlob to create a custom classifier as well as do sentiment analysis and create our rule base chatbot.


#### Textblob custom classifier

In this section we will write our custom classifier, the other Textblob features will be explained as they are implemented throughout the tutorial.

For this portion we need to create our train data. The data sets can be written in the script or imported from a file. TextBlob supports a few file formats for this operation, but we will write the data in our script.

The data is a list of tuples, each housing the training string separated by a comma followed by "pos" or "neg", representing positive or negative sentiment.

`[("phrase one", "pos"),
("phrase two", "neg")]`



Let's write our classifier module. Begin by creating a python file named twily_classifier.py, in our twily folder

We will create a function in order to import it into other scripts as a module.



- We will import the Naive Bayers classifier from TextBlob.
- Then we will create a function called trainer() which does not take any parameters at this point.
- In it, we will assign our training data list to the train variable. The train data listed below is truncated to minimize the length of this tutorial.
- Finally return NaiveBayesClassifier() constructor passing the train data as an argument.

In [10]:
from textblob.classifiers import NaiveBayesClassifier


def trainer(sentiment):
    """Trainer function for Naive Bayers classifier"""  
    return NaiveBayesClassifier(sentiment)

To test our classifier as a stand alone script, we will write test code below our trainer function.

- We test our script by calling the trainer() function
- And get the probability distribution by calling the .prob_classify() method and pass a test string we want analyzed as an argument.
- We can extract the negative value by calling the .prob() method and passing the "neg" label as an argument.

In [11]:
sentiments

[('pos',
  'I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.'),
 ('pos',
  'When I spend time with my father outside by the fire when we are camping.'),
 ('pos',
  "Every year on my kids birthday's I think back on when they were born. I remember all the sweet smells and sounds of them as a baby."),
 ('pos',
  'I came across an old keychain that my dad gave to me when I was younger. It was cool. '),
 ('pos',
  "I once went to see fireworks with my best friend. Best day of my life. We're no longer friends."),
 ('pos',
  "I'be been thinking a lot about my childhood lately_comma_ sort or reminiscing."),
 ('pos',
  'Went to the beach the other day and remember how much my friend liked going together.'),
 ('pos',
  'My husband and I went to dinner.  We were looking at photos of our kids when they were little.  It amazes me that one is already 18 and the other is just 2 short years away from being an adult also.'),


In [12]:
user_input = "I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world."
classy = trainer(sentiments).prob_classify(user_input)

In [13]:
print()
print(f'String:  {user_input} ')
print(f'---------{len(user_input)* "-"}+')
c=0
for i in set(df_mapped_empathy["empathy"]):
    print("{0} prob: {1}".format(i,classy.prob(i)))
    c+=1
    if(c>10):
        break


String:  I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world. 
---------------------------------------------------------------------------------------------------------------------------------------+
neg prob: 0
pos prob: 0


---
# stop words
---

#### NLTK stop word set
We are going to need a list of stop words. These are commonly used filler words that we want filtered out from the user input. Later we will learn how to use the stop word set.

As NLTK was installed as one of the dependencies of TextBlob, we can use it to generate a set of stop words.

We can use Python’s interactive console to generate a set of stop words and print them to the screen, then copy and paste this set into a new python file.

`from nltk.corpus import stopwords
set(stopwords.words('english'))`

- Create a document called stop_words.py.
- In the new file add a variable called sw_list.
- Paste the set to the variable.
- Save and close the file.

By saving the set of stop words into a new python file our bot will execute a lot faster than if, everytime we process user input, the application requested the stop word list from NLTK

In [14]:
from nltk.corpus import stopwords
sw_list=set(stopwords.words('english'))

In [80]:
ar_sw_list = set(stopwords.words("arabic"))


# Sentiment analysis

We will write our chatbot application as a module, as it can be isolated and tested prior to integrating with Flask.

We are going to call our module simplebot.py, this will be the core application. The application will take one argument, the user_input as a str.

The application will do a sentiment analysis of the un edited string, and it will also normalize the string and turn keywords into a set that will be used to intersect the dialog corpus file for matching responses.

### Module setup
Lets create a file named simplebot.py, in the twily folder, let's start by importing the libraries.

- twily_classifier is our TextBlob trained sentiment classifier.
- stop_words is the list of words to exclude from our input string.
- json to open the conversation corpus generated with ChatterBot.


In [15]:
""" Simple rule based chatbot with emotional monitoring """

from textblob import TextBlob
import twily_classifier as cl
import stop_words as stopwords
import json

Below the imports, start by opening our JSON file with the  trained `conversation` corpus. using the built-in `open()` function to access the file and load it into the `array` variable.

We can then call the `array` index `conversation`, it holds all the corpus data.

In [16]:
with open('EnglishChatBot.json', 'r') as f:
    array = json.load(f)

CONVERSATION = array["conversations"]

Lets go ahead and set up a few other constants and variables.

- `BOT_NAME` constant (string type) holds the name of our bot.
- `STOP_WORDS` constant (set type) holds a list of all the stop words. This was the set we generated using NLTK.
- `neg_distribution` variable (list type) holds the negative sentiment floating point value. This will be used to monitor the user's sentiment index. It will be appended everytime there is user input, with the negative probability percent value.

In [17]:
BOT_NAME = 'Twily'
STOP_WORDS = stopwords.sw_list
neg_distribution = []

Our `simplebot.py` module is made up of three functions, `sentiment()`, `simplebot()` and `escalation()`, where `escalation()` is the main function while the other two are auxiliary dependencies.

#### The sentiment() function
This function appends the `neg_distribution` list with negative probability and returns the appended value.

- From twily_classifier call the cl.trainer() function and the .prob_classify(u_input) passing our input string when called. Assign it to blob_it variable.
- From the returned value in blob_it we are going to extract just the negative values, rounded up to two decimal points and assigned to the npd variable.
- With the most recent negative value, we are going to update our neg_distribution list.
- We also return the appended value in case it is needed for another operation.

This function will be called by our main function `escalation()`.



In [18]:
def sentiment(u_input):
    """Auxiliary function: Appends 'neg_distribution'
    with negative probability also returns Negative Probability"""

    blob_it = cl.trainer().prob_classify(u_input)
    npd = round(blob_it.prob("neg"), 2)
    neg_distribution.append(npd)
    return npd

### The simplebot() function

This function implements a rule-based bot. It takes the user input in the form of a string and in sequence it preprocesses the input string, converts it to lowercase, tokenizes it and removes stop words. 

It then iterates through `CONVERSATION`, if filtered_input intersects response_set is updated. if the set is empty, it returns a message, else it returns the longest string in the set.

#### User input

- Define the function, def simplebot(user):
- Here we take the user input and turn it into a TextBlob object.

#### Pre-processing and normalization
Once we have a textblob object we can modify it by using textblob built in tools

- We normalize our input string by turning it into an all lower case string.
> `user_blob.lower()`.
- We then tokenize the textblob object into words by calling:
>`lower_input.words`.
- Finally we create a list of words not listed in `STOP_WORDS` from our textblob object and assign it to the `filtered_input` variable.

#### Set iterator
We are going to create an empty set to be updated with all the possible matches returned by our set intersection of user input and `CONVERSATION`.

- Assign an empty set to `response_set`
- Create a for loop to iterate through every list in `CONVERSATION`
- Add a nested loop to iterate through every `sentence` in each `list`.
- Turn each `sentence` into a list of words by using the `.split()` method. Assign it to the `sentence_split` variable
- If the `set()` of `filtered_input` **intersects** `sentence_split`
> - Update `response_set` with the intersection


#### Returned value based on response_set
We want to return one value from the response_set, it could be empty, with one value or more.

- If `response_set` is empty we want to return a string
>- `"I am sorry, I don't have an answer, ask again"`
- If the set has one value or more we will return the longest value
>- `return max(response_set, key=len)`



Although returning the **longest** value seems arbitrary, in our case it works, most correct answers will be the longest, but there is room for error. In our case some arbitrary errors are desirable as we need to increase the emotional index of the users.

In [38]:
import random
def simplebot(user_input):
    """Rule base bot, takes an argument, user input in form of a string. (truncated)"""
    # user input
    user_blob = TextBlob(user_input)
    
    # pre-processing and normalization
    lower_input = user_blob.lower()
    token_input = lower_input.words
    filtered_input = [w for w in token_input if w not in STOP_WORDS]
    
    # Set iterator
    response_set = set()
    for con_list in CONVERSATION:
        for sentence in con_list:
            sentence_split = sentence.split()
            if set(filtered_input).intersection(sentence_split):
                response_set.update(con_list)
    
    # Returned value based on response_set
    if not response_set:
        return "I am sorry, I don't have an answer. Ask again please!"
    else:
        return max(response_set, key=len)
#         return(random.sample(response_set,1))
    

### The escalation() function

This function takes an argument `user_input` in the form of a string and calls `sentiment()` to monitor the user sentiment index. If the emotional index, set by `sentiment()` and taken from `neg_distribution`, increases above a set threshold and is sustained, an automatic response/action is triggered. The function also sends `user_input` to `simplebot()` to generate a chatbot response.

- `live_rep` represents the trigger action taken if the emotional conditions are met for escalation.
- We pass the `user_input` to `sentiment()`, to analyse the negative sentiment of the sentence.
- We calculate the length of `neg_distribution` by using the `len()` function and assign it to `list_len`
- We send the input_string to `simplebot()` to get a response
- Create a condition, if the `list_len` is greater than `3`
> - Take the last three items in the list
>> -  If the first item of the last three is greater than .40 and greater or equal to the next two items take action by triggering `live_rep`
> - If not, `return bot_response`
- If none of the conditions above are met, `return bot_response`

In [20]:
def escalation(user_input):
    """ Takes an argument, user_input, in form of a string ..."""

    live_rep = f"We apologize {BOT_NAME} is unable to assist you, we are getting a live representative for you, please stay with us ..."

    sentiment(user_input)
    list_len = len(neg_distribution)
    bot_response = simplebot(user_input)
    if list_len > 3:
        last_3 = neg_distribution[-3:]
        if last_3[0] > .40 and last_3[0] <= last_3[1] <= last_3[2]:
            return live_rep
        else:
            return bot_response
    else:
        return bot_response

#### Stand-alone testing
In order to test our script, write a stand-alone test block that runs the chatbot as follows:

- `while True` we are going to `try` to run our program.
- In case of an `except`we end the program.

Here we can test our application as we write it and before transforming it into a web service with Flask.

By printing `neg_distribution` we are able to see the negative index on every response and see the application trigger on the last three values of the list. In our final WhatsApp application we won't display the emotional values.

In [87]:
while True:
    try:
        user_input = input('You: ')
        print(escalation(user_input))
        print(neg_distribution)
    except (KeyboardInterrupt, EOFError, SystemExit):
        break

You: Ali
I am sorry, I don't have an answer. Ask again please!
[0.41]
You: what is your name?
You're right_comma_ sorry for putting words in your mouth. And yeah_comma_ I don't know how to engage with them either. As a college student_comma_ there are plenty that kind of stalk the campus and ask for spare change. But I have little spending money_comma_ so I always have to turn them down
[0.41, 0.41]
You: I have never cheated on my wife
It's like a fighting game_comma_ but some people call it a "party game"_comma_ but basically it's a fighting game with all kinds of Nintendo characters. You have stuff like Mario_comma_ Donkey Kong_comma_ Link_comma_ Zelda_comma_ Pikachu_comma_ etc etc. So every time a new game comes out_comma_ people post online about who they want to see in it next. This new game is filled with characters and I think most people will be happy. The first game came out in 1999. Time sure flies.
[0.41, 0.41, 0.24]
You: bye
My childhood was mostly in the 1970s
[0.41, 0.41,

In [40]:
print(utterances[50])
print(utterances[51])

Got rejected from a place I wanted to work_comma_ not once but three times 
I am sorry to hear that. I hope you find a better opportunity. Did you know why they rejected you?


In [39]:
while True:
    try:
        user_input = input('You: ')
        print("Chatbot: {0}".format(escalation(user_input)))
        print(neg_distribution)
    except (KeyboardInterrupt, EOFError, SystemExit):
        break

You: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.
Chatbot: We apologize Twily is unable to assist you, we are getting a live representative for you, please stay with us ...
[0.48, 0.48, 0.41, 0.41, 0.48, 0.48, 0.48, 0.48, 0.48, 0.48]
